In [26]:
import math
import networkx as nx
import random
from datetime import datetime

In [4]:
G = nx.Graph()
graph = open("road-euroroad.txt","r")
# importance priority queue
imp_pq = list()
order = 0

In [5]:

def Connect(G, graph):
    # get number of nodes and edges
    graph_parameters = graph.readline().split()
    vertices_number = int(graph_parameters[0],base=10)
    edges_number = int(graph_parameters[1],base=10)
    print(vertices_number)
    print(edges_number)
    # add nodes to networkx graph
    for i in range(vertices_number):
        G.add_node(i+1, contracted = False,imp=0,level=0,contr_neighbours=0)
    # add edges to networkx graph
    edge = graph.readline()
    while edge:
        edge_parameters = edge.split()
        source_node = int(edge_parameters[0], base=10)
        target_node = int(edge_parameters[1], base=10)
        edge_weight = 1
        found_exist = False
        for i in G[source_node]:
            # already store the edge with different weight, choose the min weight
            if i == target_node:
                G[source_node][target_node]['weight'] = min(G[source_node][target_node]['weight'],edge_weight)
                found_exist = True
                break
        if not found_exist:
            G.add_edge(source_node,target_node,weight=edge_weight)
        edge = graph.readline()
    

In [6]:
def SetOrder(G, imp_pq, n):
    # initialize imp_pq
    for i in range(n):
        imp_pq.append((GetImportance(G,i+1),i+1))
        # print("settled node %d"%(i+1))
    order = 1
    # for i in range(1000):
    #     print(imp_pq[i+1])
    lazy_update_counter = 0
    contracted_num = 0
    completed_rate = 0
    k = 0
    print("initializing importance queue settled.")
    while len(imp_pq)>0:
        # find current lowest importance node in imp_pq
        curr_node_imp_pair = min(imp_pq, key= lambda pair:pair[0])
        curr_node  = curr_node_imp_pair[1]   
        imp_pq.remove(curr_node_imp_pair)
        # get new importance for current lowest importance node
        new_imp = GetImportance(G,curr_node)
        # print("get new importance for node %d"%curr_node)
        # lazy update
        if((len(imp_pq) == 0) or (new_imp - min(imp_pq,key=lambda pair:pair[0])[0] <= 10) or (lazy_update_counter >= 5)):
            # print("update for node %d"%curr_node)
            lazy_update_counter = 0
            G.nodes[curr_node]['imp'] = order
            order +=1
            # contract node
            G.nodes[curr_node]['contracted'] = True
            ContractNode(G,curr_node,n)
            # print("already contracted node %d"%curr_node)
            contracted_num += 1
            k +=1
            if(contracted_num == n):
                print("contracted completed!")
            if(k == 100):
                completed_rate = contracted_num / n
                print(completed_rate)
                k = 0
        else:
            imp_pq.append((new_imp,curr_node))
            lazy_update_counter +=1
            # print("recalculated prority of node %d" %curr_node)
            

In [7]:
def GetImportance(G, x):
    # get number of incident edges of x
    edges_incident = len(G[x])
    # get number of added shortcut when simulate contracting node x
    shortcuts = 0
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            if((G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                shortcuts +=1
    edge_difference = shortcuts - edges_incident
    return edge_difference + 2*G.nodes[x]['contr_neighbours'] + G.nodes[x]['level']

In [8]:
def ContractNode(G, x, n):
    mx = GetMaxEdge(G, x)
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            if ((G.nodes[i]['contracted'] == True) or (G.nodes[j]['contracted'])):
                continue
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            Check_Witness(G, n, i, x, mx)
            # print("check witness completed")
    # update importance term in incident node
    for i in G[x]:
        G.nodes[i]['contr_neighbours'] +=1
        G.nodes[i]['level'] = max(G.nodes[i]['level'], G.nodes[x]['level'] + 1)
            

In [9]:
def GetMaxEdge(G, x):
    ret = 0
    for i in G[x]:
        for j in G[x]:
            if((i != j) and (G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                ret = max(ret, G[x][i]['weight'] + G[x][j]['weight'])
    return ret

In [10]:
def Check_Witness(G, n, u, x, mx, type=None):
    # dijkstra priority queue for search witness path from u to v, excludes x
    # v is incident edge of x, excludes u
    D_pq = list()
    # initialize D_pq
    D_pq.append((0, u))
    # distance dictionary from u to any node in search tree
    D_dist = dict()
    # initialize D_dist
    D_dist[u] = 0
    # maximum iteration round for dijkstra search
    iter = int(250 * (n - order) / n)
    while((len(D_pq) > 0) and (iter > 0)):
        iter -=1
        curr_dist_pair = min(D_pq, key= lambda pair:pair[0])
        curr_dist = curr_dist_pair[0]
        a = curr_dist_pair[1]
        D_pq.remove(curr_dist_pair)
        if(curr_dist > D_dist[a]):
            continue
        for p in G[a]:
            new_dist = curr_dist + G[a][p]['weight']
            # p must not be x and not be contracted
            if(p != x and (G.nodes[p]['contracted'] == False)):
                # p must not be settled node or distance greater than new_dist
                if((p not in D_dist) or (D_dist[p] > new_dist)):
                    # prune when witness path greater than mx
                    if(p not in D_dist):
                        if new_dist < mx:
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
                    else:
                        if(D_dist[p] < mx):
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
    for v in G[x]:
        # v can not be u and not be contracted
        if ((v!=u) and (G.nodes[v]['contracted'] == False)):
            new_w = G[u][x]['weight'] + G[x][v]['weight']
            # print("%d %d %d"%(u,v,new_w))
            if((v not in D_dist) or (D_dist[v] > new_w)):
                # add shortcut
                # try:
                #     if(u,v) in G.edges:
                #         print("run here: no more add_edge")
                #         continue
                # except:
                G.add_edge(u,v,weight=new_w)
                # print("run here: add_edge:%d %d"%(u,v))
            

In [11]:
def GetDistance(G, s, t):
    # search with bi-dijkstra with ordering rank
    # initializing dijkstra from source node s
    SP_s = dict()
    parent_s = dict()
    unrelaxed_s = list()
    for node in G.nodes():
        SP_s[node] = math.inf
        parent_s[node] = None
        unrelaxed_s.append(node)
    SP_s[s] = 0
    # dijkstra forward
    while unrelaxed_s:
        node = min(unrelaxed_s, key= lambda node:SP_s[node])
        unrelaxed_s.remove(node)
        if SP_s[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_s[node] + G[node][child]['weight']
            # relax edge
            if distance < SP_s[child]:
                SP_s[child] = distance
                parent_s[child] = node
    # initializing dijkstra from target node t
    SP_t = dict()
    parent_t = dict()
    unrelaxed_t = list()
    for node in G.nodes():
        SP_t[node] = math.inf
        parent_t[node] = None
        unrelaxed_t.append(node)
    SP_t[t] = 0

    # dijkstra backward
    while unrelaxed_t:
        node = min(unrelaxed_t, key= lambda node: SP_t[node])
        unrelaxed_t.remove(node)
        if SP_t[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_t[node] + G[node][child]['weight']
            if distance < SP_t[child]:
                SP_t[child] = distance
                parent_t[child] = node
    minimum = math.inf
    merge_node = None
    for i in SP_s:
        if SP_t[i] == math.inf:
            continue
        if SP_t[i] + SP_s[i] < minimum:
            minimum = SP_s[i] + SP_t[i]
            merge_node = i
    return minimum, merge_node, SP_s, SP_t, parent_s, parent_t

In [12]:
# see the route from origin of dijkstra to a given node
def Route_dijkstra(parent, node):
    route = []
    while node != None:
        route.append(node)
        node = parent[node]
    return route[::-1]

In [13]:
def See_full_route(G, s, t):
    minimum, merge_node, SP_s, SP_t, parent_s, parent_t = GetDistance(G, s, t)
    print("shortest distance between source node %d to target node %d:"%(s,t))
    if minimum == math.inf:
        print("no path between source node %d to target node %d"%(s,t))
        return
    has_path_list.append([s,t])
    print(minimum)
    route_from_source = Route_dijkstra(parent_s, merge_node)
    # show route
    print("route from source node %d:"%s)
    print(route_from_source)
    route_from_target = Route_dijkstra(parent_t, merge_node)
    # show route
    print("route from target node %d:"%t)
    print(route_from_target)
    route = route_from_source + route_from_target[::-1][1:]
    # show route
    print("entire route:")
    print(route)
    unvisited = 0
    for s_node, s_dist in SP_s.items():
        for t_node, t_dist in SP_t.items():
            if s_node == t_node and s_dist == t_dist == math.inf:
                unvisited += 1
    print(f"""we have skipped {unvisited} nodes from a graph with {len(G)}, 
    so we have skipped {unvisited/len(G)*100}% of the nodes in our search space.""")
    print("\n\n")

In [14]:
Connect(G, graph)

1174
1417


In [15]:
edges_before = [*G.edges()]

In [16]:
# G.nodes.data()
for i in range(100):
    print(i+1,G.nodes[i+1])

1 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
2 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
3 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
4 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
5 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
6 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
7 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
8 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
9 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
10 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
11 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
12 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
13 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
14 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
15 {'contracted': False, 'imp

In [17]:
SetOrder(G, imp_pq, len(G.nodes))

initializing importance queue settled.
0.08517887563884156
0.17035775127768313
0.2555366269165247
0.34071550255536626
0.42589437819420783
0.5110732538330494
0.596252129471891
0.6814310051107325
0.7666098807495741
0.8517887563884157
0.9369676320272572
contracted completed!


In [18]:
edges_after = [*G.edges()]
print("# edges before", len(edges_before))
print("# edges after", len(edges_after))

# edges before 1417
# edges after 2524


In [19]:
added_edges = list(set(edges_after) - set(edges_before))
added_edges

[(825, 829),
 (321, 323),
 (254, 256),
 (766, 768),
 (321, 781),
 (69, 876),
 (188, 246),
 (141, 421),
 (401, 735),
 (536, 591),
 (141, 145),
 (587, 590),
 (238, 910),
 (181, 1031),
 (188, 236),
 (280, 896),
 (101, 669),
 (107, 141),
 (987, 989),
 (270, 578),
 (409, 683),
 (8, 482),
 (197, 241),
 (687, 693),
 (47, 671),
 (305, 309),
 (317, 781),
 (39, 874),
 (768, 772),
 (550, 910),
 (851, 853),
 (36, 393),
 (440, 530),
 (1118, 1122),
 (238, 722),
 (506, 508),
 (195, 1118),
 (193, 196),
 (321, 324),
 (81, 86),
 (95, 98),
 (69, 416),
 (768, 937),
 (269, 745),
 (401, 428),
 (259, 465),
 (629, 903),
 (117, 152),
 (305, 943),
 (77, 107),
 (7, 137),
 (440, 442),
 (4, 390),
 (236, 429),
 (194, 725),
 (297, 515),
 (563, 580),
 (317, 946),
 (445, 448),
 (145, 149),
 (115, 194),
 (246, 265),
 (188, 193),
 (39, 42),
 (240, 910),
 (221, 228),
 (884, 1074),
 (284, 314),
 (343, 347),
 (137, 173),
 (206, 209),
 (49, 51),
 (297, 320),
 (280, 401),
 (998, 1003),
 (109, 144),
 (107, 416),
 (429, 433),


In [20]:
query_list = list()
i = 0
while i < 100:
    a = random.randint(1,len(G.nodes))
    b = random.randint(1,len(G.nodes))
    pair = sorted((a,b))
    if (a==b or (pair in query_list)):
        continue
    query_list.append(pair)
    i +=1
query_list

[[818, 827],
 [6, 737],
 [304, 545],
 [549, 873],
 [54, 146],
 [90, 724],
 [557, 887],
 [127, 922],
 [458, 953],
 [634, 1020],
 [47, 1171],
 [86, 625],
 [128, 1150],
 [377, 379],
 [260, 491],
 [26, 456],
 [309, 630],
 [48, 410],
 [103, 326],
 [191, 890],
 [327, 392],
 [187, 251],
 [241, 1026],
 [363, 864],
 [638, 1026],
 [525, 763],
 [48, 1055],
 [399, 695],
 [98, 116],
 [763, 1084],
 [145, 1013],
 [269, 386],
 [191, 327],
 [497, 1055],
 [432, 1041],
 [71, 99],
 [146, 902],
 [7, 467],
 [10, 943],
 [675, 951],
 [453, 529],
 [29, 829],
 [706, 870],
 [432, 452],
 [300, 920],
 [318, 353],
 [320, 844],
 [824, 1091],
 [530, 555],
 [71, 584],
 [90, 565],
 [168, 697],
 [382, 781],
 [571, 753],
 [434, 744],
 [208, 436],
 [54, 64],
 [521, 663],
 [379, 801],
 [170, 181],
 [257, 964],
 [196, 745],
 [944, 1025],
 [330, 830],
 [614, 943],
 [23, 636],
 [501, 1159],
 [629, 1045],
 [60, 468],
 [291, 346],
 [286, 1062],
 [112, 647],
 [111, 227],
 [770, 827],
 [410, 441],
 [351, 553],
 [115, 546],
 [920,

In [23]:
for i in range(100):
    See_full_route(G,query_list[i][0],query_list[i][1])

shortest distance between source node 818 to target node 827:
5
route from source node 818:
[818, 819, 600]
route from target node 827:
[827, 828, 829, 600]
entire route:
[818, 819, 600, 829, 828, 827]
we have skipped 1157 nodes from a graph with 1174, 
    so we have skipped 98.55195911413969% of the nodes in our search space.



shortest distance between source node 6 to target node 737:
12
route from source node 6:
[6, 7, 401]
route from target node 737:
[737, 504, 465, 401]
entire route:
[6, 7, 401, 465, 504, 737]
we have skipped 1127 nodes from a graph with 1174, 
    so we have skipped 95.99659284497444% of the nodes in our search space.



shortest distance between source node 304 to target node 545:
22
route from source node 304:
[304, 280]
route from target node 545:
[545, 188, 236, 280]
entire route:
[304, 280, 236, 188, 545]
we have skipped 1152 nodes from a graph with 1174, 
    so we have skipped 98.12606473594549% of the nodes in our search space.



shortest distance bet

In [22]:
G_original = nx.Graph()
graph = open("road-euroroad.txt","r")
Connect(G_original,graph)
has_path_list = list()

1174
1417


In [65]:
def See_full_route_constract(G, s, t):
    minimum, merge_node, SP_s, SP_t, parent_s, parent_t = GetDistance(G, s, t)
    # print("shortest distance between source node %d to target node %d:"%(s,t))
    if minimum == math.inf:
        print("no path between source node %d to target node %d"%(s,t))
        return
    # has_path_list.append([s,t])
    # print(minimum)
    route_from_source = Route_dijkstra(parent_s, merge_node)
    # show route
    # print("route from source node %d:"%s)
    # print(route_from_source)
    route_from_target = Route_dijkstra(parent_t, merge_node)
    # show route
    # print("route from target node %d:"%t)
    # print(route_from_target)
    route = route_from_source + route_from_target[::-1][1:]
    # show route
    # print("entire route:")
    # print(route)
    # unvisited = 0
    # for s_node, s_dist in SP_s.items():
    #     for t_node, t_dist in SP_t.items():
    #         if s_node == t_node and s_dist == t_dist == math.inf:
    #             unvisited += 1
    # print(f"""we have skipped {unvisited} nodes from a graph with {len(G)}, 
    # so we have skipped {unvisited/len(G)*100}% of the nodes in our search space.""")
    # print("\n\n")
    # print([minimum, route])

In [66]:
def dijkstra_with_contraction(G, source, destination, contracted = None):
    nx.set_node_attributes(G, {contracted: True}, 'contracted')
    
        
    shortest_path = dict()
    heap = list()
    
    for i in G.nodes():
        if not nx.get_node_attributes(G, 'contracted')[i]:
            shortest_path[i] = math.inf
            heap.append(i)
    shortest_path[source] = 0
    
    while len(heap)>0:
        q = min(heap, key= lambda node : shortest_path[node])
        if q == destination:
            nx.set_node_attributes(G, {contracted: False}, 'contracted')
            return shortest_path[q]
        heap.remove(q)
        # G[q] are incident edges of q
        for v in G[q]:
            # if the node is contracted, skip it
            if not nx.get_node_attributes(G, 'contracted')[v]:
                distance = shortest_path[q] + G[q][v]['weight']
                if distance < shortest_path[v]:
                    shortest_path[v] = distance
    nx.set_node_attributes(G, {contracted: False}, 'contracted')
    
    # can not reach the destination
    return math.inf

In [74]:
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra
sd_a = datetime.datetime.now()
for i in range(len(query_list)):
    distance = dijkstra_with_contraction(G_original, query_list[i][0], query_list[i][1])
sd_b = datetime.datetime.now()
((sd_b - sd_a).total_seconds())/len(query_list)

1.4417042599999998

In [75]:
ch_a = datetime.datetime.now()
for i in range(len(query_list)):
    See_full_route_constract(G, query_list[i][0], query_list[i][1])
ch_b = datetime.datetime.now()
((ch_b - ch_a).total_seconds())/len(query_list)

no path between source node 127 to target node 922
no path between source node 86 to target node 625
no path between source node 128 to target node 1150
no path between source node 377 to target node 379
no path between source node 187 to target node 251
no path between source node 48 to target node 1055
no path between source node 269 to target node 386
no path between source node 497 to target node 1055
no path between source node 382 to target node 781
no path between source node 379 to target node 801
no path between source node 60 to target node 468
no path between source node 112 to target node 647
no path between source node 126 to target node 398
no path between source node 752 to target node 1093
no path between source node 2 to target node 970
no path between source node 58 to target node 804
no path between source node 423 to target node 1054


0.00838953